In [19]:
import os
import csv
import pandas as pd
from acquire import *
import re

# Robot Pickles!

In [20]:
def read_pickles_in_directory(directory_path):
    robot_dict = {}
    regex_pattern = r'(.+)_((match_history)|(stats))\.pkl'

    for filename in os.listdir(directory_path):
        match = re.match(regex_pattern, filename)
        if match:
            robot_name = match.group(1)
            table_type = match.group(2)
            key = robot_name

            with open(os.path.join(directory_path, filename), 'rb') as f:
                df = pd.read_pickle(f)
                if not df.empty:
                    if key in robot_dict:
                        robot_dict[key][table_type] = df
                    else:
                        robot_dict[key] = {table_type: df}

    return robot_dict

# Read All Pickle Files

In [21]:
robot_dict = read_pickles_in_directory("pickle-files")

# Verify Pickle Files

#### Robot Links JSON

In [22]:
# loading the JSON file with all season & robot links
robot_links_dict = load_from_json("all_season_robot_links.json")

In [24]:
total_length = 0
for value in robot_links_dict.values():
    if isinstance(value, list):
        total_length += len(value)

print(f"Total number of items in all lists: {total_length}")

Total number of items in all lists: 394


In [25]:
# Set the directory where the files are located
directory = "pickle-files"

# Initialize sets to store robot names from match history and stats tables
match_history_robots = set()
stats_robots = set()

# Iterate over files in the directory
for filename in os.listdir(directory):
    # Check if the file is a match history table
    if filename.endswith("_match_history.pkl"):
        # Extract the robot name from the filename
        robot_name = filename[:-len("_match_history.pkl")]
        # Add the robot name to the set of match history robots
        match_history_robots.add(robot_name)
    # Check if the file is a stats table
    elif filename.endswith("_stats.pkl"):
        # Extract the robot name from the filename
        robot_name = filename[:-len("_stats.pkl")]
        # Add the robot name to the set of stats robots
        stats_robots.add(robot_name)

# Get the set of unique robot names from both tables
unique_robots = match_history_robots.union(stats_robots)

# Initialize lists to store missing robot names
missing_match_history = []
missing_stats = []
missing_both = []
missing_from_directory = []

# Check each robot name in the unique set
for robot_name in unique_robots:
    # Check if the robot name is in the robot_links_dict
    robot_name_in_links = False
    for links_list in robot_links_dict.values():
        for link in links_list:
            if robot_name in link:
                robot_name_in_links = True
                break
        if robot_name_in_links:
            break
    if not robot_name_in_links:
        missing_from_directory.append(robot_name)
    else:
        # Check if the robot has a match history table
        if robot_name not in match_history_robots:
            missing_match_history.append(robot_name)
        # Check if the robot has a stats table
        if robot_name not in stats_robots:
            missing_stats.append(robot_name)
        # Check if the robot has both tables
        if robot_name not in match_history_robots or robot_name not in stats_robots:
            missing_both.append(robot_name)

# Check each robot name in the robot_links_dict
for key in robot_links_dict.keys():
    for robot_link in robot_links_dict[key]:
        robot_name = robot_link.split("/")[-2]
        # Check if the robot name is in the set of unique robots
        if robot_name not in unique_robots:
            missing_from_directory.append(robot_name)

# Print the results
print("Total unique robots in directory:", len(unique_robots))
for robot_name in missing_match_history:
    print(f"{robot_name} missing match history table")
for robot_name in missing_stats:
    print(f"{robot_name} missing stats table")
for robot_name in missing_both:
    print(f"{robot_name} missing match history and/or stats table")

print("Missing these robots in directory:")
print("="*30)

missing_robot_links_list = []
for robot_name in missing_from_directory:
    # Check each robot link in the robot_links_dict
    for key in robot_links_dict.keys():
        for robot_link in robot_links_dict[key]:
            # Extract the robot name from the link
            link_robot_name = robot_link.split("/")[-2]
            # Check if the link robot name matches the missing robot name
            if link_robot_name == robot_name:
                # Print the missing robot name and link
                print(f"{robot_name}\n{robot_link}")
                missing_robot_links_list.append(robot_link)
                print("="*30)
                break

Total unique robots in directory: 316
Missing these robots in directory:
jager-2021
https://battlebots.com/robot/jager-2021/
smeeeeeeeeeeeeeeeeee-2021
https://battlebots.com/robot/smeeeeeeeeeeeeeeeeee-2021/
electric-ray-2019
https://battlebots.com/robot/electric-ray-2019/
bombshell-2
https://battlebots.com/robot/bombshell-2/
brutus-2018
https://battlebots.com/robot/brutus-2018/
chomp-s3
https://battlebots.com/robot/chomp-s3/
double-dutch
https://battlebots.com/robot/double-dutch/
duck
https://battlebots.com/robot/duck/
free-shipping-2018
https://battlebots.com/robot/free-shipping-2018/
gemini-2
https://battlebots.com/robot/gemini-2/
huge-s3
https://battlebots.com/robot/huge-s3/
hypothermia-s3
https://battlebots.com/robot/hypothermia-s3/
kraken-s3
https://battlebots.com/robot/kraken-s3/
lucky
https://battlebots.com/robot/lucky/
minotaur-s3
https://battlebots.com/robot/minotaur-s3/
monsoon-s3
https://battlebots.com/robot/monsoon-s3/
parallax-s3
https://battlebots.com/robot/parallax-s3/
p

In [27]:
# Set the output CSV file name
output_file = "missing_robot_links.csv"

# Write the missing robot links to the CSV file
with open(output_file, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    for link in missing_robot_links_list:
        writer.writerow([link])

# Dictionary of DataFrames!

#### Keys = Robot

In [28]:
for i in robot_dict.keys():
    print(i)

tombstone-2021
bronco-2020
rampage-2020
doomba-wcvii
free-shipping-2021
rotator-2019
nightmare-2
deadlift-2020
extinguisher-2019
lock-jaw-wcvii
mammoth-2019
black-dragon-wcvii
uppercut-2020
lucky-2019
madcatter-2021
jackpot-2021
beta-2020
railgun-max
nightmare
deathroll-2019
rotator-2020
tantrum
shatter-2020
bombshell-2019
valkyrie-2019
creepy-crawlies
icewave-2021
valkyrie-wcvii
radioactive
hypershock-2020
cobalt-wcvii
triple-crown-2021
starchild-wcvii
whiplash-2020
bale-spear-s3
monsoon-2019
hydra-wcvii
tombstone-2
black-dragon-2020
dragon-slayer-2021
copperhead-2021
splatter
foxtrot-2019
overhaul-wcvii
cobalt-2021
perfect-phoenix-2021
skorpios-2019
bloodsport-2020
hijinx-wcvii
battlesaw-2019
gigabyte-2020
tantrum-wcvii
ghost-raptor-2021
bite-force-2
copperhead-2019
sow
sawblaze
photon-storm-s2
kingpin-2019
terrortops-wcvii
overhaul
daisy-cutter-2019
madcatter-wcvii
mammoth-2021
ultimo-destructo
tombstone-2019
double-tap-wcvii
perfect-phoenix-2020
gruff-2021
petunia
stinger-2
ominous

#### Nested Keys = DataFrames of Tables

In [29]:
robot_dict['rotator-2019'].keys()

dict_keys(['match_history', 'stats'])

# DataFrame # 1

In [30]:
robot_dict['rotator-2019']['match_history']

,Season,Round,Matchup,Results
0,WCVII,Episode 703,Rotator vs Hydra,Hydra by KO 1m24s
1,WCVII,Episode 706,Rotator vs Jackpot,Rotator by KO 2m49s
2,WCVII,Episode 711,Rotator vs Bloodsport,Bloodsport by KO 1m47s
3,WCVII,Episode 713,Rotator vs Fusion,NaN
4,2021,Episode 602,Kraken vs Rotator,Rotator by JD
5,2021,Episode 606,Rotator vs Black Dragon,Rotator by JD
6,2021,Episode 612,Rotator vs Captain Shrederator,Rotator by KO 1m16s
7,2021,Episode 613,Rotator vs Tantrum,Tantrum 2m59s
8,2021,SCSF 3,Glitch vs Rotator,Glitch by KO 52s
9,2020,Bounty Bronco,Bronco vs Rotator,Rotator by JD


# DataFrame # 2

In [31]:
robot_dict['rotator-2019']['stats']

,Stats,Career,WCVII,2021,2020,2019,2018,2016
0,Total matches,29,3,5,9,5,6,1
1,Win percentage,55%,33%,60%,67%,40%,67%,0%
2,Total wins,16,1,3,6,2,4,0
3,Losses,13,2,2,3,3,2,1
4,Knockouts,9,1,1,4,2,1,0
5,KO percentage,31%,33%,20%,44%,40%,17%,0%
6,Average knockout time,109 s,169 s,76 s,114 s,118 s,67 s,NaN
7,Knockouts against,5,2,1,0,0,1,1
8,KO against percentage,17%,67%,20%,0%,0%,17%,100%
9,Judges decision wins,11,0,2,6,0,3,0
